# 简介
本文介绍了使用PaddleX对布匹瑕疵检测模型快速训练、使用Paddle-Lite将训练得到的工业质检模型快速部署到Android手机的方法。

- 关于工业质检和布匹瑕疵检测数据集的详细介绍请参考该系列文章的第一部分：[CascadeRCNN和YOLOv3_Enhance的布匹瑕疵检测模型训练部署](https://aistudio.baidu.com/aistudio/projectdetail/532715)
- 关于部署移动端迁移学习模型的详细过程请参考：[手把手教你部署移动端迁移学习模型（PaddleX、Paddle-Lite）](https://aistudio.baidu.com/aistudio/projectdetail/613622)


## 关于本项目

> 针对项目还存在的改进空间，以及其它模型在不同移动设备的部署，希望大家多交流观点、介绍经验，共同学习进步。[个人主](https://aistudio.baidu.com/aistudio/personalcenter/thirdview/90149)

# 拉取工具库

In [ ]:
!unzip data/data11768/guangdong1_round1_train1_20190818.zip -d data/
!unzip data/data11768/guangdong1_round1_testA_20190818.zip -d data/

In [ ]:
!pip install paddlelite
!pip install paddlex

In [ ]:
!git clone -b release/0.2 https://gitee.com/paddlepaddle/PaddleDetection.git

Cloning into 'PaddleDetection'...
remote: Enumerating objects: 5704, done.
remote: Counting objects: 100% (5704/5704), done.
remote: Compressing objects: 100% (2684/2684), done.
remote: Total 5704 (delta 4109), reused 4070 (delta 2943), pack-reused 0
Receiving objects: 100% (5704/5704), 20.43 MiB | 1.75 MiB/s, done.
Resolving deltas: 100% (4109/4109), done.
Checking connectivity... done.


# 数据集准备

In [ ]:
defect_name2label = {
    '破洞': 1, '水渍': 2, '油渍': 2, '污渍': 2, '三丝': 3, '结头': 4, '花板跳': 5, '百脚': 6, '毛粒': 7,
    '粗经': 8, '松经': 9, '断经': 10, '吊经': 11, '粗维': 12, '纬缩': 13, '浆斑': 14, '整经结': 15, '星跳': 16, '跳花': 16,
    '断氨纶': 17, '稀密档': 18, '浪纹档': 18, '色差档': 18, '磨痕': 19, '轧痕': 19, '修痕': 19, '烧毛痕': 19, '死皱': 20, '云织': 20,
    '双纬': 20, '双经': 20, '跳纱': 20, '筘路': 20, '纬纱不良': 20,
}
# defect_name2label = {
#     '破洞': 1, '水渍': 2, '油渍': 3, '污渍': 4, '三丝': 5, '结头': 6, '花板跳': 7, '百脚': 8, '毛粒': 9,
#     '粗经': 10, '松经': 11, '断经': 12, '吊经': 13, '粗维': 14, '纬缩': 15, '浆斑': 16, '整经结': 17, '星跳': 18, '跳花': 19,
#     '断氨纶': 20, '稀密档': 21, '浪纹档': 22, '色差档': 23, '磨痕': 24, '轧痕': 25, '修痕': 26, '烧毛痕': 27, '死皱': 28, '云织': 29,
#     '双纬': 30, '双经': 31, '跳纱': 32, '筘路': 33, '纬纱不良': 34,
# }

## 创建coco格式训练集

In [ ]:
class Fabric2COCO:

    def __init__(self,mode="train2017"):
    # def __init__(self,mode="val"):
        self.images = []
        self.annotations = []
        self.categories = []
        self.img_id = 0
        self.ann_id = 0
        self.mode =mode
        # if not os.path.exists("coco/images/{}".format(self.mode)):
        #     os.makedirs("coco/images/{}".format(self.mode))
        if not os.path.exists("PaddleDetection/dataset/coco/{}".format(self.mode)):
            os.makedirs("PaddleDetection/dataset/coco/{}".format(self.mode))

    def to_coco(self, anno_file,img_dir):
        self._init_categories()
        anno_result= pd.read_json(open(anno_file,"r"))
        anno_result = anno_result.head(int(anno_result['name'].count()*0.9))
        # anno_result = anno_result.tail(int(anno_result['name'].count()*0.1)+1)        
        name_list=anno_result["name"].unique()
        for img_name in name_list:
            img_anno = anno_result[anno_result["name"] == img_name]
            bboxs = img_anno["bbox"].tolist()
            defect_names = img_anno["defect_name"].tolist()
            assert img_anno["name"].unique()[0] == img_name

            img_path=os.path.join(img_dir,img_name)
            # img =cv2.imread(img_path)
            # h,w,c=img.shape
            h,w=1000,2446
            self.images.append(self._image(img_path,h, w))

            self._cp_img(img_path)

            for bbox, defect_name in zip(bboxs, defect_names):
                label= defect_name2label[defect_name]
                annotation = self._annotation(label, bbox)
                self.annotations.append(annotation)
                self.ann_id += 1
            self.img_id += 1
        instance = {}
        instance['info'] = 'fabric defect'
        instance['license'] = ['none']
        instance['images'] = self.images
        instance['annotations'] = self.annotations
        instance['categories'] = self.categories
        return instance

    def _init_categories(self):
        for v in range(1,21):
            print(v)
            category = {}
            category['id'] = v
            category['name'] = str(v)
            category['supercategory'] = 'defect_name'
            self.categories.append(category)
        # for k, v in defect_name2label.items():
        #     category = {}
        #     category['id'] = v
        #     category['name'] = k
        #     category['supercategory'] = 'defect_name'
        #     self.categories.append(category)

    def _image(self, path,h,w):
        image = {}
        image['height'] = h
        image['width'] = w
        image['id'] = self.img_id
        image['file_name'] = os.path.basename(path)
        return image

    def _annotation(self,label,bbox):
        area=(bbox[2]-bbox[0])*(bbox[3]-bbox[1])
        points=[[bbox[0],bbox[1]],[bbox[2],bbox[1]],[bbox[2],bbox[3]],[bbox[0],bbox[3]]]
        annotation = {}
        annotation['id'] = self.ann_id
        annotation['image_id'] = self.img_id
        annotation['category_id'] = label
        annotation['segmentation'] = [np.asarray(points).flatten().tolist()]
        annotation['bbox'] = self._get_box(points)
        annotation['iscrowd'] = 0
        annotation['area'] = area
        return annotation

    def _cp_img(self, img_path):
        shutil.copy(img_path, os.path.join("PaddleDetection/dataset/coco/{}".format(self.mode), os.path.basename(img_path)))
        # shutil.copy(img_path, os.path.join("coco/images/{}".format(self.mode), os.path.basename(img_path)))
    def _get_box(self, points):
        min_x = min_y = np.inf
        max_x = max_y = 0
        for x, y in points:
            min_x = min(min_x, x)
            min_y = min(min_y, y)
            max_x = max(max_x, x)
            max_y = max(max_y, y)
        '''coco,[x,y,w,h]'''
        return [min_x, min_y, max_x - min_x, max_y - min_y]
    def save_coco_json(self, instance, save_path):
        import json
        with open(save_path, 'w') as fp:
            json.dump(instance, fp, indent=1, separators=(',', ': '))

In [ ]:
'''转换有瑕疵的样本为coco格式'''
img_dir = "data/guangdong1_round1_train1_20190818/defect_Images"
anno_dir="data/guangdong1_round1_train1_20190818/Annotations/anno_train.json"
fabric2coco = Fabric2COCO()
train_instance = fabric2coco.to_coco(anno_dir,img_dir)
if not os.path.exists("PaddleDetection/dataset/coco/annotations/"):
    os.makedirs("PaddleDetection/dataset/coco/annotations/")
fabric2coco.save_coco_json(train_instance, "PaddleDetection/dataset/coco/annotations/"+'instances_{}.json'.format("train2017"))
# fabric2coco.save_coco_json(train_instance, "coco/annotations/"+'instances_{}.json'.format("val"))

## 创建coco格式验证集

In [ ]:
class Fabric2COCO:

    def __init__(self,mode="val2017"):
        self.images = []
        self.annotations = []
        self.categories = []
        self.img_id = 0
        self.ann_id = 0
        self.mode =mode
        # if not os.path.exists("coco/images/{}".format(self.mode)):
        #     os.makedirs("coco/images/{}".format(self.mode))
        if not os.path.exists("PaddleDetection/dataset/coco/{}".format(self.mode)):
            os.makedirs("PaddleDetection/dataset/coco/{}".format(self.mode))

    def to_coco(self, anno_file,img_dir):
        self._init_categories()
        anno_result= pd.read_json(open(anno_file,"r"))
        anno_result = anno_result.tail(int(anno_result['name'].count()*0.1)+1)        
        name_list=anno_result["name"].unique()
        for img_name in name_list:
            img_anno = anno_result[anno_result["name"] == img_name]
            bboxs = img_anno["bbox"].tolist()
            defect_names = img_anno["defect_name"].tolist()
            assert img_anno["name"].unique()[0] == img_name

            img_path=os.path.join(img_dir,img_name)
            # img =cv2.imread(img_path)
            # h,w,c=img.shape
            h,w=1000,2446
            self.images.append(self._image(img_path,h, w))

            self._cp_img(img_path)

            for bbox, defect_name in zip(bboxs, defect_names):
                label= defect_name2label[defect_name]
                annotation = self._annotation(label, bbox)
                self.annotations.append(annotation)
                self.ann_id += 1
            self.img_id += 1
        instance = {}
        instance['info'] = 'fabric defect'
        instance['license'] = ['none']
        instance['images'] = self.images
        instance['annotations'] = self.annotations
        instance['categories'] = self.categories
        return instance

    def _init_categories(self):
        for v in range(1,21):
            print(v)
            category = {}
            category['id'] = v
            category['name'] = str(v)
            category['supercategory'] = 'defect_name'
            self.categories.append(category)
        # for k, v in defect_name2label.items():
        #     category = {}
        #     category['id'] = v
        #     category['name'] = k
        #     category['supercategory'] = 'defect_name'
        #     self.categories.append(category)

    def _image(self, path,h,w):
        image = {}
        image['height'] = h
        image['width'] = w
        image['id'] = self.img_id
        image['file_name'] = os.path.basename(path)
        return image

    def _annotation(self,label,bbox):
        area=(bbox[2]-bbox[0])*(bbox[3]-bbox[1])
        points=[[bbox[0],bbox[1]],[bbox[2],bbox[1]],[bbox[2],bbox[3]],[bbox[0],bbox[3]]]
        annotation = {}
        annotation['id'] = self.ann_id
        annotation['image_id'] = self.img_id
        annotation['category_id'] = label
        annotation['segmentation'] = [np.asarray(points).flatten().tolist()]
        annotation['bbox'] = self._get_box(points)
        annotation['iscrowd'] = 0
        annotation['area'] = area
        return annotation

    def _cp_img(self, img_path):
        shutil.copy(img_path, os.path.join("PaddleDetection/dataset/coco/{}".format(self.mode), os.path.basename(img_path)))
        # shutil.copy(img_path, os.path.join("coco/images/{}".format(self.mode), os.path.basename(img_path)))
    def _get_box(self, points):
        min_x = min_y = np.inf
        max_x = max_y = 0
        for x, y in points:
            min_x = min(min_x, x)
            min_y = min(min_y, y)
            max_x = max(max_x, x)
            max_y = max(max_y, y)
        '''coco,[x,y,w,h]'''
        return [min_x, min_y, max_x - min_x, max_y - min_y]
    def save_coco_json(self, instance, save_path):
        import json
        with open(save_path, 'w') as fp:
            json.dump(instance, fp, indent=1, separators=(',', ': '))

In [ ]:
'''转换有瑕疵的样本为coco格式'''
img_dir = "data/guangdong1_round1_train1_20190818/defect_Images"
anno_dir="data/guangdong1_round1_train1_20190818/Annotations/anno_train.json"
fabric2coco = Fabric2COCO()
train_instance = fabric2coco.to_coco(anno_dir,img_dir)
if not os.path.exists("PaddleDetection/dataset/coco/annotations/"):
    os.makedirs("PaddleDetection/dataset/coco/annotations/")
fabric2coco.save_coco_json(train_instance, "PaddleDetection/dataset/coco/annotations/"+'instances_{}.json'.format("val2017"))

## 整理后最终文件目录格式

  ```
  ./coco/
  ├── annotations
  │   ├── instances_train2014.json
  │   ├── instances_train2017.json
  |   ...
  ├── train2017
  │   ├── 5cac654fb8e699da1546312852.jpg
  │   ├── 14d68415c6f020021536006312.jpg
  |   ...
  ├── val2017
  │   ├── 6cf97c00f1a180120939190444.jpg
  │   ├── cb42508af46ed0a50817439434.jpg
  |   ...

## 数据集相关计算
参考[CascadeRCNN和YOLOv3_Enhance的布匹瑕疵检测模型训练部署](https://aistudio.baidu.com/aistudio/projectdetail/532715)，这里直接给出计算结果
```python
[[  7.70564186  31.008     ]
 [ 13.42273099  12.16      ]
 [570.46606705 141.056     ]
 [  8.69991823 594.016     ]
 [ 45.48814391  20.064     ]
 [  4.22567457  37.696     ]
 [ 36.0425184  297.312     ]
 [  5.46852003  94.24      ]
 [  4.72281276  15.2       ]]
 
Accuracy: 60.76%

Boxes:
 [  7.70564186  13.42273099 570.46606705   8.69991823  45.48814391
   4.22567457  36.0425184    5.46852003   4.72281276]-[ 31.008  12.16  141.056 594.016  20.064  37.696 297.312  94.24   15.2  ]
   
Ratios:
 [0.01, 0.06, 0.11, 0.12, 0.25, 0.31, 1.1, 2.27, 4.04]
```

# 使用FasterRCNN训练（PaddleX）
> 注意：用PaddleX的YoloV3模型进行训练时，一直无法收敛，因此本文仅演示使用`faster_rcnn_r50_vd`快速训练迁移学习模型。
## 配置GPU环境

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import paddlex as pdx

## 数据处理与数据增强

In [ ]:
from paddlex.det import transforms
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.Normalize(),
    transforms.ResizeByShort(short_size=608, max_size=608),
    transforms.Padding(coarsest_stride=32)
])

eval_transforms = transforms.Compose([
    transforms.Normalize(),
    transforms.ResizeByShort(short_size=608, max_size=608),
    transforms.Padding(coarsest_stride=32),
])

## 创建数据读取器

In [ ]:
train_dataset = pdx.datasets.CocoDetection(
    data_dir='PaddleDetection/dataset/coco/train2017',
    ann_file='PaddleDetection/dataset/coco/annotations/instances_train2017.json',
    transforms=train_transforms,
    shuffle=True)
eval_dataset = pdx.datasets.CocoDetection(
    data_dir='PaddleDetection/dataset/coco/val2017',
    ann_file='PaddleDetection/dataset/coco/annotations/instances_val2017.json',
    transforms=eval_transforms)

loading annotations into memory...
Done (t=0.04s)
creating index...
index created!
2020-07-20 18:04:48 [INFO]	Starting to read file list from dataset...
2020-07-20 18:04:48 [INFO]	4290 samples in file PaddleDetection/dataset/coco/annotations/instances_train2017.json
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
2020-07-20 18:04:48 [INFO]	Starting to read file list from dataset...
2020-07-20 18:04:48 [INFO]	484 samples in file PaddleDetection/dataset/coco/annotations/instances_val2017.json


In [ ]:
num_classes = len(train_dataset.labels) + 1
model = pdx.det.FasterRCNN(backbone='ResNet50_vd',with_fpn=False,num_classes=num_classes)

In [ ]:
model.train(
    num_epochs=12,
    train_dataset=train_dataset,
    train_batch_size=4,
    eval_dataset=eval_dataset,
    learning_rate=0.0125,
    lr_decay_epochs=[8, 11],
    save_dir='output/faster_rcnn_r50_vd',
    log_interval_steps=200,
    use_vdl=True)

In [ ]:
model.evaluate(eval_dataset, batch_size=1, epoch_id=None, metric=None, return_details=False)

2020-07-12 09:44:19 [INFO]	Start to evaluating(total_samples=484, total_steps=484)...


100%|██████████| 484/484 [00:59<00:00,  8.15it/s]


creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.46s).
Accumulating evaluation results...
DONE (t=0.18s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.138
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.341
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.086
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.093
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.103
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.118
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.198
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.258
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.260
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.173
 Average Recall     (AR) @[ IoU=0.50

OrderedDict([('bbox_mmap', 0.1378795392258368)])

## 模型效果评估
![file](https://ai-studio-static-online.cdn.bcebos.com/23d4bda7075a43f197c9c6c37efd3abb73103e4451b44fea857734155951ac2a)


In [ ]:
eval_details_file = 'output/faster_rcnn_r50_vd/epoch_12/eval_details.json'
pdx.det.draw_pr_curve(eval_details_file, save_dir='./fabric_flaw_det')

creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.55s).
Accumulating evaluation results...
DONE (t=0.09s).


## 模型预测

In [ ]:
import paddlex as pdx
predictor = pdx.deploy.Predictor('./inference_model/faster_rcnn_r50_vd')
result = predictor.predict(image='PaddleDetection/dataset/coco/val2017/9a49f25c5d9b6e390840112792.jpg')

In [ ]:
print(result)

[{'category_id': 4, 'bbox': [192.10128784179688, 191.18878173828125, 15.174957275390625, 22.388519287109375], 'score': 0.1731969267129898, 'category': '4'}, {'category_id': 5, 'bbox': [118.27978515625, 401.5374755859375, 2327.72021484375, 326.073486328125], 'score': 0.9391258358955383, 'category': '5'}, {'category_id': 5, 'bbox': [0.0, 525.7344970703125, 1528.667724609375, 246.27685546875], 'score': 0.9079427123069763, 'category': '5'}, {'category_id': 5, 'bbox': [898.4940185546875, 371.75079345703125, 1547.5059814453125, 260.0858154296875], 'score': 0.8581957817077637, 'category': '5'}]


In [ ]:
pdx.det.visualize('PaddleDetection/dataset/coco/val2017/9a49f25c5d9b6e390840112792.jpg', result, threshold=0.9, save_dir='./output/faster_rcnn_r50_vd')

2020-07-20 18:05:55 [INFO]	The visualized result is saved as ./output/faster_rcnn_r50_vd/visualize_9a49f25c5d9b6e390840112792.jpg


![file](https://ai-studio-static-online.cdn.bcebos.com/cb547c105145430ca0524339e02554ef208e8460ba6c4f9baee2a65720119b96)

## 模型导出
> 注意：这里`faster_rcnn_r50_vd`可以顺利完成模型优化opt，而在[手把手教你部署移动端迁移学习模型（PaddleX、Paddle-Lite）](https://aistudio.baidu.com/aistudio/projectdetail/613622)一文中曾介绍过`faster_rcnn_r50_fpn`模型优化失败，原因是paddle升级了generate_proposals op支持RpnRoisLod输出，lite还没有及时更新。

In [ ]:
!paddlex --export_inference --model_dir=output/faster_rcnn_r50_vd/best_model --save_dir=./inference_model/faster_rcnn_r50_vd

In [ ]:
!paddle_lite_opt --print_model_ops=true  --model_dir=inference_model/faster_rcnn_r50_vd --valid_targets=npu,arm

OPs in the input model include:
                                        OP_name      Host       X86      CUDA       ARM    OpenCL      FPGA       NPU       XPU     RKNPU       APU       Any       Unk
                                 affine_channel                                       Y                                                                                
                               anchor_generator                                       Y                                                                                
                                       box_clip                                       Y                                                                                
                                      box_coder                                       Y         Y                                                                      
                                         conv2d                   Y         Y         Y         Y         Y                     

In [ ]:
# 使用华为NPU预测时的设置
!paddle_lite_opt \
    --model_file=inference_model/faster_rcnn_r50_vd/__model__ \
    --param_file=inference_model/faster_rcnn_r50_vd/__params__ \
    --optimize_out_type=naive_buffer \
    --optimize_out=mobile_npu_model \
    --valid_targets=npu,arm 

# 使用YOLOv3模型训练（PaddleDetection）
> 此处可以点击右上角【代码执行器—重启执行器】先释放显存，然后再继续运行后续cell，避免报错

## 修改以下文件中的分类数
- PaddleDetection/configs/yolov3_darknet.yml
- PaddleDetection/ppdet/modeling/anchor_heads/yolo_head.py
- PaddleDetection/ppdet/data/reader.py
- PaddleDetection/ppdet/data/transform/batch_operators.py
> 注意：如果训练时打印的分类数仍然没有改过来，需要重启一下环境

参考配置文件
- `/home/aistudio/PaddleDetection/configs/yolov3_darknet.yml`
```
architecture: YOLOv3
use_gpu: true
max_iters: 150000
log_smooth_window: 200
save_dir: output
snapshot_iter: 10000
metric: COCO
pretrain_weights: https://paddle-imagenet-models-name.bj.bcebos.com/DarkNet53_pretrained.tar
weights: output/yolov3_darknet/model_final
num_classes: 20
use_fine_grained_loss: false

YOLOv3:
  backbone: DarkNet
  yolo_head: YOLOv3Head

DarkNet:
  norm_type: sync_bn
  norm_decay: 0.
  depth: 53

YOLOv3Head:
  anchor_masks: [[6, 7, 8], [3, 4, 5], [0, 1, 2]]
  anchors: [[36, 297], [5, 94], [5, 15],
                [9, 594], [45, 20], [4, 38],
                [8, 31], [13, 12], [570, 141]]
  norm_decay: 0.
  yolo_loss: YOLOv3Loss
  nms:
    background_label: -1
    keep_top_k: 100
    nms_threshold: 0.45
    nms_top_k: 1000
    normalized: false
    score_threshold: 0.01

YOLOv3Loss:
  # batch_size here is only used for fine grained loss, not used
  # for training batch_size setting, training batch_size setting
  # is in configs/yolov3_reader.yml TrainReader.batch_size, batch
  # size here should be set as same value as TrainReader.batch_size
  batch_size: 8
  ignore_thresh: 0.7
  label_smooth: true

LearningRate:
  base_lr: 0.000125
  schedulers:
  - !PiecewiseDecay
    gamma: 0.1
    milestones:
    - 80000
    - 140000
  - !LinearWarmup
    start_factor: 0.
    steps: 4000

OptimizerBuilder:
  optimizer:
    momentum: 0.9
    type: Momentum
  regularizer:
    factor: 0.0005
    type: L2

_READER_: 'yolov3_reader.yml'
```
- `/home/aistudio/PaddleDetection/configs/yolov3_reader.yml`
```
TrainReader:
  inputs_def:
    fields: ['image', 'gt_bbox', 'gt_class', 'gt_score']
    num_max_boxes: 50
  dataset:
    !COCODataSet
      image_dir: train2017
      anno_path: annotations/instances_train2017.json
      dataset_dir: dataset/coco
      with_background: false
  sample_transforms:
    - !DecodeImage
      to_rgb: True
      with_mixup: True
    - !MixupImage
      alpha: 1.5
      beta: 1.5
    - !ColorDistort {}
    - !RandomExpand
      fill_value: [123.675, 116.28, 103.53]
    - !RandomCrop {}
    - !RandomFlipImage
      is_normalized: false
    - !NormalizeBox {}
    - !PadBox
      num_max_boxes: 50
    - !BboxXYXY2XYWH {}
  batch_transforms:
  - !RandomShape
    sizes: [320, 352, 384, 416, 448, 480, 512, 544, 576, 608]
    random_inter: True
  - !NormalizeImage
    mean: [0.3914940814114446, 0.3605475730949753, 0.36263685530651174]
    std: [0.11077973580477549, 0.10994100883809227, 0.10480770290045718]
    is_scale: True
    is_channel_first: false
  - !Permute
    to_bgr: false
    channel_first: True
  # Gt2YoloTarget is only used when use_fine_grained_loss set as true,
  # this operator will be deleted automatically if use_fine_grained_loss
  # is set as false
  - !Gt2YoloTarget
    anchor_masks: [[6, 7, 8], [3, 4, 5], [0, 1, 2]]
    anchors: [[10, 13], [16, 30], [33, 23],
              [30, 61], [62, 45], [59, 119],
              [116, 90], [156, 198], [373, 326]]
    downsample_ratios: [32, 16, 8]
  batch_size: 16
  shuffle: true
  mixup_epoch: 250
  drop_last: true
  worker_num: 8
  bufsize: 16
  use_process: true


EvalReader:
  inputs_def:
    fields: ['image', 'im_size', 'im_id']
    num_max_boxes: 50
  dataset:
    !COCODataSet
      image_dir: val2017
      anno_path: annotations/instances_val2017.json
      dataset_dir: dataset/coco
      with_background: false
  sample_transforms:
    - !DecodeImage
      to_rgb: True
    - !ResizeImage
      target_size: 608
      interp: 2
    - !NormalizeImage
      mean: [0.3914940814114446, 0.3605475730949753, 0.36263685530651174]
      std: [0.11077973580477549, 0.10994100883809227, 0.10480770290045718]
      is_scale: True
      is_channel_first: false
    - !PadBox
      num_max_boxes: 50
    - !Permute
      to_bgr: false
      channel_first: True
  batch_size: 8
  drop_empty: false
  worker_num: 8
  bufsize: 16

TestReader:
  inputs_def:
    image_shape: [3, 608, 608]
    fields: ['image', 'im_size', 'im_id']
  dataset:
    !ImageFolder
      anno_path: annotations/instances_val2017.json
      with_background: false
  sample_transforms:
    - !DecodeImage
      to_rgb: True
    - !ResizeImage
      target_size: 608
      interp: 2
    - !NormalizeImage
      mean: [0.3914940814114446, 0.3605475730949753, 0.36263685530651174]
      std: [0.11077973580477549, 0.10994100883809227, 0.10480770290045718]
      is_scale: True
      is_channel_first: false
    - !Permute
      to_bgr: false
      channel_first: True
  batch_size: 1
```

## 模型训练

In [ ]:
!cd cocoapi/PythonAPI && make install

In [ ]:
# 如果训练时出现No module named ‘pycocotools’报错，卸载pycocotools库再重装
!pip uninstall pycocotools -y
!pip install pycocotools

In [ ]:
%set_env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [1]:
%cd /home/aistudio/PaddleDetection/

/home/aistudio/PaddleDetection


In [ ]:
%run tools/train.py -c configs/yolov3_darknet.yml --eval

In [2]:
%run tools/infer.py -c configs/yolov3_darknet.yml \
                    --infer_img=dataset/coco/val2017/9a49f25c5d9b6e390840112792.jpg \
                    --output_dir=output/ \
                    --draw_threshold=0.5 \
                    -o weights=output/yolov3_darknet/model_final

2020-07-20 18:33:07,562-INFO: Loading parameters from output/yolov3_darknet/model_final...
2020-07-20 18:33:10,285-INFO: Not found annotation file annotations/instances_val2017.json, load coco17 categories.
2020-07-20 18:33:10,540-INFO: Infer iter 0
2020-07-20 18:33:10,593-INFO: Detection bbox results save in output/9a49f25c5d9b6e390840112792.jpg


![file](https://ai-studio-static-online.cdn.bcebos.com/f373e787122b4eb59ebb0f0fc0ab80ebbd0dc67525284a39b75f8e83b61acbfa)


## 模型导出
训练得到一个满足要求的模型后，如果想要将该模型接入到C++预测库或者Serving服务，需要通过tools/export_model.py导出该模型。
[参考链接](https://github.com/PaddlePaddle/PaddleDetection/blob/release/0.2/docs/advanced_tutorials/inference/EXPORT_MODEL.md)

In [ ]:
# 导出YOLOv3模型
%run tools/export_model.py -c configs/yolov3_darknet.yml \
        --output_dir=./inference_model \
        -o weights=output/yolov3_darknet/model_final

2020-07-18 19:08:20,346-INFO: Loading parameters from output/yolov3_darknet/model_final...
2020-07-18 19:08:23,078-INFO: save_inference_model pruned unused feed variables im_id
2020-07-18 19:08:23,080-INFO: Export inference model to ./inference_model/yolov3_darknet, input: ['image', 'im_size'], output: ['multiclass_nms_0.tmp_0']...


In [ ]:
!paddle_lite_opt \
    --model_file=inference_model/yolov3_darknet/__model__ \
    --param_file=inference_model/yolov3_darknet/__params__ \
    --optimize_out_type=naive_buffer \
    --optimize_out=model \
    --valid_targets=npu,arm 

## 移动端部署
- Android开发环境：Android Studio on Ubuntu 18.04 64-bit
- 移动端设备：华为Mate20 Pro
- 部署过程参考：[手把手教你部署移动端迁移学习模型（PaddleX、Paddle-Lite）](https://aistudio.baidu.com/aistudio/projectdetail/613622)
	- 要准备的模型文件`model.nb`和参考标签文件`fabricflaw-labels.txt`***（注意：标签不支持中文）***
    ```
    background
    hole
    dirty
    three yarn
    knot
    slab yarn
    centipede
    neps
    coarse warp
    loosen warp
    broken warp
    hanged warp
    coarse pick
    looped weft
    stain
    knot warp
    broken spandex
    thin thick place
    smash
    others
    ```
	- 修改`PaddleLite-android-demo/yolo_detection_demo/app/src/main/res/values/strings.xml`
	![file](https://ai-studio-static-online.cdn.bcebos.com/fc95e799a1aa4c158d798e09916809654aa5caaff3dd47be9fab754fd6d8e92d)


# 小结
- 即使比较复杂的目标检测场景，用PaddleX的FasterRCNN进行快速迁移学习的效果还是不错的
- PaddleDetection模型库要比PaddleX丰富很多，但是dcn、fpn这些算子目前Paddle-Lite并不支持，因此面向移动端部署时，选择比较有限
- 在要求检测速度和小目标检测准确度兼具的工业质检场景，目前在Paddle-Lite-Demo中选择比较有限，正如官方文档所说，本文使用的`yolov3_darknet`更适合服务端的部署，如果不使用PaddleSlim，预测速度会掉到 1FPS 左右；而使用`mobilenet`做`backbone`时，又会降低准确率。